# ApacheSpark - ML - NoSupervisado

<p><strong>Objetivo: </strong> El objetivo de este cuaderno es aplicar el algoritmo de Kmeans sobre un conjunto de datos ejemplo:</p>

## Importando librerias y cargando los datos

In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

Para este ejercicio se estará utilizando el conjunto de datos sample_kmeans_data, son datos que representan números asociados a distintas características, solo se tienen seis filas, sin buscar un significado en especial de sus características sino con el objetivo de ejemplificar el código para clustering:

In [0]:
# Loads data.
df = spark.read.format("libsvm").load("/FileStore/tables/sample_kmeans_data.txt")
display(df)

label,features
0.0,"Map(vectorType -> sparse, length -> 3, indices -> List(), values -> List())"
1.0,"Map(vectorType -> sparse, length -> 3, indices -> List(0, 1, 2), values -> List(0.1, 0.1, 0.1))"
2.0,"Map(vectorType -> sparse, length -> 3, indices -> List(0, 1, 2), values -> List(0.2, 0.2, 0.2))"
3.0,"Map(vectorType -> sparse, length -> 3, indices -> List(0, 1, 2), values -> List(9.0, 9.0, 9.0))"
4.0,"Map(vectorType -> sparse, length -> 3, indices -> List(0, 1, 2), values -> List(9.1, 9.1, 9.1))"
5.0,"Map(vectorType -> sparse, length -> 3, indices -> List(0, 1, 2), values -> List(9.2, 9.2, 9.2))"


## Configurar el modelo k-means

En este paso se configura el modelo de agrupamiento a los datos anteriores, se define K=2 para encontrar dos grupos. Se ejecuta el método .fit para obtener el modelo.

In [0]:
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(df)

Se pueden ver los valores asociados a cada parámetro con el siguiente código:

In [0]:
print(kmeans.explainParams())

distanceMeasure: the distance measure. Supported options: 'euclidean' and 'cosine'. (default: euclidean)
featuresCol: features column name. (default: features)
initMode: The initialization algorithm. This can be either "random" to choose random points as initial cluster centers, or "k-means||" to use a parallel variant of k-means++ (default: k-means||)
initSteps: The number of steps for k-means|| initialization mode. Must be > 0. (default: 2)
k: The number of clusters to create. Must be > 1. (default: 2, current: 2)
maxIter: max number of iterations (>= 0). (default: 20)
predictionCol: prediction column name. (default: prediction)
seed: random seed. (default: 7969353092125344463, current: 1)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 0.0001)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)


## Obteniendo los clusters

Con el método .transform se pueden obtener el cluster asociado a cada elemento del conjunto de datos

In [0]:
predictions = model.transform(df)

In [0]:
display(predictions)

label,features,prediction
0.0,"Map(vectorType -> sparse, length -> 3, indices -> List(), values -> List())",1
1.0,"Map(vectorType -> sparse, length -> 3, indices -> List(0, 1, 2), values -> List(0.1, 0.1, 0.1))",1
2.0,"Map(vectorType -> sparse, length -> 3, indices -> List(0, 1, 2), values -> List(0.2, 0.2, 0.2))",1
3.0,"Map(vectorType -> sparse, length -> 3, indices -> List(0, 1, 2), values -> List(9.0, 9.0, 9.0))",0
4.0,"Map(vectorType -> sparse, length -> 3, indices -> List(0, 1, 2), values -> List(9.1, 9.1, 9.1))",0
5.0,"Map(vectorType -> sparse, length -> 3, indices -> List(0, 1, 2), values -> List(9.2, 9.2, 9.2))",0


## Evaluando la calidad de los clusters

Para evaluar la calidad de los clusters primero se debe crear un Evaluador:

In [0]:
evaluator = ClusteringEvaluator()

Una de las métricas utilizadas para evaluar la calidad de los clusters es el Índice de la Silueta:

In [0]:
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.9997530305375207


Si se desea ver el centroide de cada cluster, para evaluar las características principales de cada grupo, se puede calcular de la siguiente manera:

In [0]:
# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[9.1 9.1 9.1]
[0.1 0.1 0.1]
